In [ ]:
!uv pip install guardrails-ai==0.5.15

In [ ]:
from jinja2 import Template

prompt_template = """
  Extraire les informations des accords d'entreprise relatif aux heures supplémentaires à partir du texte suivant et le renvoyer sous la forme d'un objet JSON comme ceci :
  {
    "base_legale_hebdomadaire": "",
    "duree_travail_annuelle_heures": "",
    "contingent_annuel_heures_supplementaires": "",
    "premier_taux_majoration": "",
    "plage_premier_taux_majoration": "",
    "deuxieme_taux_majoration": "",
    "plage_deuxieme_taux_majoration": "",
    "troisieme_taux_majoration": "",
    "plage_troisieme_taux_majoration": "",
    "presence_repos_compensateur_remplacement": "",
    "taux_majoration_contrepartie_obligatoire_en_repos": "",
    "delai_prevenance": ""
  }

  Text:
  {{ user_input }}
"""

user_input = """
Pour les 4 premières heures (36ème à 39ème heure) : 20 % ;
Pour les 3 heures suivantes (40ème à 43ème heure) : 30 % ;
À partir de la 44ème heure : 50 %.
[...] le contingent annuel d’heures supplémentaires est fixé à 250 heures par salarié.
Les heures supplémentaires effectuées et comptabilisées [...] sont compensées, avec leur majoration, pour moitié en heures supplémentaires payées et pour moitié par du repos compensateur.
4 heures supplémentaires =  droit à 2 x 1,20 = 2,4 heures de repos compensateur de remplacement, l’autre moitié des heures supplémentaires effectuées étant rémunérée, avec la majoration applicable.

"""

# Render prompt
template = Template(prompt_template)
prompt = template.render(user_input=user_input)

print(prompt)


In [4]:
import subprocess
from guardrails import Guard

# Define your RAIL spec
rail_spec = """
<rail version="0.1">
  <output>
    <string name="base_legale_hebdomadaire" description="Base légale hebdomadaire, 35 heures"/>
    <string name="duree_travail_annuelle_heures" description="Durée annuelle d'heures de travail, 1607 heures"/>
    <string name="contingent_annuel_heures_supplementaires" description="Nombre d'heures au contingent annuel d'heures supplémentaires"/>
    <string name="premier_taux_majoration" description="Premier taux de majoration"/>
    <string name="plage_premier_taux_majoration" description="Plage des heures du premier taux de majoration"/>
    <string name="deuxieme_taux_majoration" description="Deuxième taux de majoration"/>
    <string name="plage_deuxieme_taux_majoration" description="Plage des heures du deuxième taux de majoration"/>
    <string name="troisieme_taux_majoration" description="Troisième taux de majoration"/>
    <string name="plage_troisieme_taux_majoration" description="Plage des heures du troisième taux de majoration"/>
    <string name="presence_repos_compensateur_remplacement" description="Mention à un repos compensateur de remplacement (RCR)"/>
    <string name="taux_majoration_contrepartie_obligatoire_en_repos" description="Taux de majoration de la contrepartie obligatoire en repos (COR)"/>
    <string name="delai_prevenance" description="Délai de prévenance des heures supplémentaires"/>
  </output>
  <prompt>
  Extraire les informations des accords d'entreprise relatif aux heures supplémentaires à partir du texte suivant et le renvoyer sous la forme d'un objet JSON comme ceci :
  {
    "base_legale_hebdomadaire": "",
    "duree_travail_annuelle_heures": "",
    "contingent_annuel_heures_supplementaires": "",
    "premier_taux_majoration": "",
    "plage_premier_taux_majoration": "",
    "deuxieme_taux_majoration": "",
    "plage_deuxieme_taux_majoration": "",
    "troisieme_taux_majoration": "",
    "plage_troisieme_taux_majoration": "",
    "presence_repos_compensateur_remplacement": "",
    "taux_majoration_contrepartie_obligatoire_en_repos": "",
    "delai_prevenance": ""
  }
  </prompt>
</rail>
"""

# Create Guard object
guard = Guard.from_rail_string(rail_spec)

# Run LLM via Ollama
result = subprocess.run(
    ["ollama", "run", "llama3"],
    input=prompt.encode(),
    capture_output=True
)

llm_output = result.stdout.decode().strip()

# Validate with Guardrails
validated = guard.parse(llm_output)
print(validated)


KeyboardInterrupt

